## ORDER:

- Acquire
- CSV & Function
- Drop Columns (remove tax to prevent leakage)
- Rename Columns
- Remove Outliers
- Remove Duplicates
- Convert Datatypes
- Split it right

- Throw into a wrangle.py w/ comments

In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password
from sklearn.model_selection import train_test_split
import sklearn.preprocessing

In [2]:
def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
def new_zillow_data():
    '''
    This function reads the Bed/Bath count, Finished Sq Ft, Taxable Value, Year Built, and Amount Taxed
    from the 2017 Properties Zillow data from the Codeup SQL server into a df.
    '''
    sql_query = """
                SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, 
                taxvaluedollarcnt, yearbuilt, fips, transactiondate
                FROM properties_2017
                JOIN propertylandusetype USING(propertylandusetypeid)
                JOIN predictions_2017 USING(parcelid)
                WHERE propertylandusetype.propertylandusetypeid = 261 AND 279;
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    
    return df

In [4]:
#                 SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, 
#                 taxvaluedollarcnt, yearbuilt, taxamount, fips, transactiondate
#                 FROM properties_2017
#                 JOIN predictions_2017 as p USING(parcelid)
#                 WHERE transactiondate < '2018-01-01' AND propertylandusetypeid LIKE '261'

In [5]:
def get_zillow_data():
    '''
    This function reads in zillow data from the Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_zillow_data()
        
        # Cache data
        df.to_csv('zillow.csv')
        
    return df

In [6]:
zillow = get_zillow_data()


In [7]:
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips,transactiondate
0,4.0,3.5,3100.0,1023282.0,1998.0,6059.0,2017-01-01
1,2.0,1.0,1465.0,464000.0,1967.0,6111.0,2017-01-01
2,3.0,2.0,1243.0,564778.0,1962.0,6059.0,2017-01-01
3,4.0,3.0,2376.0,145143.0,1970.0,6037.0,2017-01-01
4,4.0,3.0,2962.0,773303.0,1950.0,6037.0,2017-01-01


In [8]:
zillow.shape
#hit our target from the rubric

(52442, 7)

In [9]:
zillow.columns
#lets explore and rename these soon



Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'fips', 'transactiondate'],
      dtype='object')

In [10]:
zillow = zillow.rename(columns = {'bedroomcnt':'bedrooms', 
                            'bathroomcnt':'bathrooms', 
                            'calculatedfinishedsquarefeet':'area',
                            'taxvaluedollarcnt':'tax_value', 
                            'yearbuilt':'year_built',
                            'transactiondate': 'transaction_date',
                            'fips': 'county'})

In [11]:
zillow.head()

,bedrooms,bathrooms,area,tax_value,year_built,county,transaction_date
0,4.0,3.5,3100.0,1023282.0,1998.0,6059.0,2017-01-01
1,2.0,1.0,1465.0,464000.0,1967.0,6111.0,2017-01-01
2,3.0,2.0,1243.0,564778.0,1962.0,6059.0,2017-01-01
3,4.0,3.0,2376.0,145143.0,1970.0,6037.0,2017-01-01
4,4.0,3.0,2962.0,773303.0,1950.0,6037.0,2017-01-01


In [12]:
# zillow.drop(columns = 'taxamount', inplace = True)
# zillow.head()

In [13]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [14]:
#will handle later

zillow.isnull().sum()

bedrooms              0
bathrooms             0
area                 82
tax_value             1
year_built          116
county                0
transaction_date      0
dtype: int64

In [48]:
zillow.county.value_counts()

LA         33911
Orange     14136
Ventura     4395
Name: county, dtype: int64

In [49]:
33911 + 14136 + 4395


52442

In [16]:
def split_data(df):
    '''
    Takes in a dataframe and returns train, validate, and test subset dataframes. 
    '''
    train, test = train_test_split(df, test_size = .2, random_state = 222)
    train, validate = train_test_split(train, test_size = .3, random_state = 222)
    return train, validate, test


In [17]:
def scale_zillow(train, validate, test):
    '''
    Takes train, validate, test datasets as an argument and returns the dataframes with 
    tax_vale, and area scaled columns.
    '''
    ## MinMaxScaler
    scaler = sklearn.preprocessing.MinMaxScaler()

    # Fit scaler to data
    scaler.fit(train[['tax_vale', 'area']])

    # Execute scaling
    train[['area', 'tax_value_scaled']] = scaler.transform(train[['area', 'tax_value']])
    validate[['area_scaled', 'tax_value_scaled']] = scaler.transform(validate[['area', 'tax_value']])
    test[['area_scaled', 'tax_value_scaled']] = scaler.transform(test[['area', 'tax_value']])
    return train, validate, test


In [18]:
zillow.columns

Index(['bedrooms', 'bathrooms', 'area', 'tax_value', 'year_built', 'county',
       'transaction_date'],
      dtype='object')

In [19]:
zillow['county'] = zillow['county'].replace({6037.0:'LA',6059.0: 'Orange',6111.0:'Ventura'})


In [20]:
zillow.head()

,bedrooms,bathrooms,area,tax_value,year_built,county,transaction_date
0,4.0,3.5,3100.0,1023282.0,1998.0,Orange,2017-01-01
1,2.0,1.0,1465.0,464000.0,1967.0,Ventura,2017-01-01
2,3.0,2.0,1243.0,564778.0,1962.0,Orange,2017-01-01
3,4.0,3.0,2376.0,145143.0,1970.0,LA,2017-01-01
4,4.0,3.0,2962.0,773303.0,1950.0,LA,2017-01-01


In [21]:
zillow['county']

0         Orange
1        Ventura
2         Orange
3             LA
4             LA
          ...   
52437         LA
52438    Ventura
52439         LA
52440         LA
52441         LA
Name: county, Length: 52442, dtype: object

In [ ]:
# Create county mean column

## Still need to add to function

In [32]:
LA = zillow[zillow.county=='LA']
orange = zillow[zillow.county=='Orange']
ventura = zillow[zillow.county=='Ventura']

la_avg = round(LA.tax_value.mean(),2)
orange_avg = round(orange.tax_value.mean(),2)
ventura_avg = round(ventura.tax_value.mean(),2)

In [33]:
def assign_county_avg(row):
    if row['county']=='LA':
        return la_avg
    if row['county']=='Orange':
        return orange_avg
    if row['county']=='Ventura':
        return ventura_avg

In [34]:
zillow['county_tax_avg'] = zillow.apply(lambda row: assign_county_avg(row), axis = 1)

In [36]:
zillow.head()

,bedrooms,bathrooms,area,tax_value,year_built,county,transaction_date,county_tax_avg
0,4.0,3.5,3100.0,1023282.0,1998.0,Orange,2017-01-01,604685.17
1,2.0,1.0,1465.0,464000.0,1967.0,Ventura,2017-01-01,515671.00
2,3.0,2.0,1243.0,564778.0,1962.0,Orange,2017-01-01,604685.17
3,4.0,3.0,2376.0,145143.0,1970.0,LA,2017-01-01,500236.14
4,4.0,3.0,2962.0,773303.0,1950.0,LA,2017-01-01,500236.14


In [39]:
# Create baseline - needed for way later down the road
#zillow - train


# baseline = train.tax_value.median()
# train['baseline'] = baseline
# validate['baseline'] = baseline
# test['baseline'] = baseline

In [41]:
# year built to age

from datetime import date
zillow['age'] = date.today().year-zillow['year_built']

In [43]:
zillow.head()

#will need to change to int and drop year_built

,bedrooms,bathrooms,area,tax_value,year_built,county,transaction_date,county_tax_avg,age
0,4.0,3.5,3100.0,1023282.0,1998.0,Orange,2017-01-01,604685.17,23.0
1,2.0,1.0,1465.0,464000.0,1967.0,Ventura,2017-01-01,515671.00,54.0
2,3.0,2.0,1243.0,564778.0,1962.0,Orange,2017-01-01,604685.17,59.0
3,4.0,3.0,2376.0,145143.0,1970.0,LA,2017-01-01,500236.14,51.0
4,4.0,3.0,2962.0,773303.0,1950.0,LA,2017-01-01,500236.14,71.0


In [45]:
# One hot county

dummy_df = pd.get_dummies(zillow[['county']], drop_first = True)
zillow = pd.concat([zillow, dummy_df], axis = 1)

In [47]:
zillow.head()

,bedrooms,bathrooms,area,tax_value,year_built,county,transaction_date,county_tax_avg,age,county_Orange,county_Ventura,county_Orange,county_Ventura
0,4.0,3.5,3100.0,1023282.0,1998.0,Orange,2017-01-01,604685.17,23.0,1,0,1,0
1,2.0,1.0,1465.0,464000.0,1967.0,Ventura,2017-01-01,515671.00,54.0,0,1,0,1
2,3.0,2.0,1243.0,564778.0,1962.0,Orange,2017-01-01,604685.17,59.0,1,0,1,0
3,4.0,3.0,2376.0,145143.0,1970.0,LA,2017-01-01,500236.14,51.0,0,0,0,0
4,4.0,3.0,2962.0,773303.0,1950.0,LA,2017-01-01,500236.14,71.0,0,0,0,0


In [51]:
# time to update new values and push into wrangle.

In [ ]:
age_graph = sns.histplot(data=train, x="age", discrete = True)
plt.figure(figsize=(16, 6))
age_graph.set(Title = "Property Age Distribution", xlabel = "Age of the property", ylabel = "Number of Properties");

In [50]:
def regression_errors(y, y_hat):
    #calculate residuals
    residuals = y - y_hat
    
    #residuals squared
    residuals_squared = residuals ** 2
    
    #sum of squared errors
    SSE = residuals_squared.sum()
    
    #explained sum of squares
    ESS = sum((y_hat - y.mean()) ** 2)
    
    #total sum of squares
    TSS = ESS + SSE
    
    #mean of squared errors
    MSE = SSE / len(y)
    
    #root of mean of squared errors
    RMSE = MSE ** (1/2)
    
    #gimme gimme
    print('Model Metrics')
    print('=============')
    return pd.Series({
        'SSE': SSE,
        'ESS': ESS,
        'TSS': TSS,
        'MSE': MSE,
        'RMSE': RMSE
    })

In [ ]:
print('Training/In-Sample: \n')
lars = LassoLars(alpha=1.0)
lars.fit(X_train, y_train['tax_value'])
train['lars_yhat'] = lars.predict(X_train)
model.regression_errors(y_train['tax_value'], train['lars_yhat'])